# Assistants API（GPT-4）とDALL·E-3を使ったスライド作成

このノートブックは、新しい[Assistants API](https://platform.openai.com/docs/assistants/overview)（GPT-4）とDALL·E-3を使用して、情報豊富で視覚的に魅力的なスライドを作成する方法を説明しています。<br>
スライド作成は多くの職業において重要な要素ですが、労力がかかり時間を消費する作業でもあります。さらに、データから洞察を抽出し、それらをスライド上で効果的に表現することは困難な場合があります。<br><br>このクックブックレシピでは、新しいAssistants APIを活用して、Microsoft PowerPointやGoogle Slidesを使用することなく、エンドツーエンドのスライド作成プロセスを自動化する方法を実演し、貴重な時間と労力を節約できることを示します！

## 0. セットアップ

In [1]:
from IPython.display import display, Image
from openai import OpenAI
import os
import pandas as pd
import json
import io
from PIL import Image
import requests

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

#Lets import some helper functions for assistants from https://cookbook.openai.com/examples/assistants_api_overview_python
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message,file_ids=None):
    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }
    if file_ids:
        params['file_ids']=file_ids

    client.beta.threads.messages.create(
        **params
)
    return client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)


## 1. コンテンツの作成

このレシピでは、私たちの会社であるNotReal Corporationの四半期財務レビュー用の簡潔な架空のプレゼンテーションを作成します。会社の収益性に影響を与えている重要なトレンドをいくつか強調したいと思います。<br> 手元にいくつかの財務データがあるとしましょう。データを読み込んで、確認してみましょう...

In [2]:
financial_data_path = 'data/NotRealCorp_financial_data.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)


,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


ご覧のとおり、このデータには異なる販売チャネル全体の四半期収益、コスト、顧客データが含まれています。個人アナリストとして機能し、PowerPoint用の素晴らしい可視化を作成できるAssistantを作成しましょう！

まず、アシスタントがファイルにアクセスできるように、ファイルをアップロードする必要があります。

In [3]:
file = client.files.create(
  file=open('data/NotRealCorp_financial_data.json',"rb"),
  purpose='assistants',
)


これで、Assistantを作成する準備が整いました。Assistantにデータサイエンティストとして行動するよう指示し、私たちが与えるクエリを受け取って、適切なデータ可視化を出力するために必要なコードを実行させることができます。ここでのinstructionsパラメータは、ChatCompletionsエンドポイントのシステム指示に似ており、Assistantをガイドするのに役立ちます。また、Code Interpreterツールを有効にすることで、Assistantがコードをかけるようになります。最後に、使用したいファイルを指定できます。この場合は、上で作成した`financial_data`ファイルのみです。

In [4]:
assistant = client.beta.assistants.create(
  instructions="You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}],
  file_ids=[file.id]
)


それでは、スレッドを作成し、最初のリクエストとしてアシスタントに四半期利益の計算を依頼し、その後時間経過による販売チャネル別の利益をプロットしてもらいましょう。アシスタントは各四半期の利益を自動的に計算し、直接依頼しなくても四半期と年を組み合わせた新しい列も作成してくれます。また、各線の色も指定することができます。

In [5]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of the lines are green, light red, and light blue",
      "file_ids": [file.id]
    }
  ]
)


私たちのスレッドの実行を開始できます

In [6]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


これで、画像が作成されたかどうかをチェックするループを開始できます。注意：これには数分かかる場合があります

In [7]:
messages = client.beta.threads.messages.list(thread_id=thread.id)


In [8]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #See if image has been created
        messages.data[0].content[0].image_file
        #Sleep to make sure run has completed
        time.sleep(5)
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')


Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


アシスタントが追加したメッセージを見てみましょう。

In [9]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]


[MessageContentImageFile(image_file=ImageFile(file_id='file-0rKABLygI02MgwwhpgWdRFY1'), type='image_file'),
 MessageContentText(text=Text(annotations=[], value="The profit has been calculated for each distribution channel by quarter and year. Next, I'll create a line plot to visualize these profits. As specified, I will use green for the 'Online Sales', light red for 'Direct Sales', and light blue for 'Retail Partners' channels. Let's create the plot."), type='text'),
 MessageContentText(text=Text(annotations=[], value="The JSON data has been successfully restructured into a tabular dataframe format. It includes the year, quarter, distribution channel, revenue, costs, customer count, and a combined 'Time' representation of 'Year Quarter'. Now, we have the necessary information to calculate the profit (revenue minus cost) by quarter and year.\n\nTo visualize the profit across the different distribution channels with a line plot, we will proceed with the following steps:\n\n1. Calculate 

アシスタントからの最後のメッセージ（最新のメッセージが最初に表示される）に、私たちが探している画像ファイルが含まれていることがわかります。ここで興味深い点は、最初の解析が失敗したため、アシスタントがJSONデータの解析を数回試行できたことです。これはアシスタントの適応性を示しています。

In [10]:
# Quick helper function to convert our output file to a png
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)


In [11]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "../images/NotRealCorp_chart.png"
convert_file_to_png(plot_file_id,image_path)

#Upload
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)


プロットを読み込みましょう！

![The Image](../images/NotRealCorp_chart.png)

素晴らしい！たった一文で、アシスタントにcode interpreterを使用して収益性を計算し、さまざまな流通チャネルの3つの線グラフを描画させることができました。<br><br>
これでスライド用の素敵なビジュアルができましたが、それに合わせていくつかの洞察も欲しいところです。

## 2. インサイトの生成

画像からインサイトを得るには、スレッドに新しいメッセージを追加するだけです。Assistantはメッセージ履歴を使用して、提供されたビジュアルから簡潔な要点を教えてくれます。

In [12]:
submit_message(assistant.id,thread,"Give me two medium length sentences (~20-30 words per sentence) of the \
      most important insights from the plot you just created.\
             These will be used for a slide deck, and they should be about the\
                     'so what' behind the data."
)


Run(id='run_NWoygMcBfHUr58fCE4Cn6rxN', assistant_id='asst_3T362kLlTyAq0FUnkvjjQczO', cancelled_at=None, completed_at=None, created_at=1701827074, expires_at=1701827674, failed_at=None, file_ids=['file-piTokyHGllwGITzIpoG8dok3'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_73TgtFoJMlEJvb13ngjTnAo3', tools=[ToolAssistantToolsCode(type='code_interpreter')])

実行が完了すると、最新のメッセージを確認できます

In [13]:
# Hard coded wait for a response, as the assistant may iterate on the bullets.
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)


The plot reveals a consistent upward trend in profits for all distribution channels, indicating successful business growth over time. Particularly, 'Online Sales' shows a notable increase, underscoring the importance of digital presence in revenue generation.


素晴らしい！アシスタントはオンライン売上利益の注目すべき成長を特定し、これが大きなデジタルプレゼンスの重要性を示していることを推測できました。では、スライドの魅力的なタイトルを作成しましょう。

In [14]:
submit_message(assistant.id,thread,"Given the plot and bullet points you created,\
 come up with a very brief title for a slide. It should reflect just the main insights you came up with."
)


Run(id='run_q6E85J31jCw3QkHpjJKl969P', assistant_id='asst_3T362kLlTyAq0FUnkvjjQczO', cancelled_at=None, completed_at=None, created_at=1701827084, expires_at=1701827684, failed_at=None, file_ids=['file-piTokyHGllwGITzIpoG8dok3'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_73TgtFoJMlEJvb13ngjTnAo3', tools=[ToolAssistantToolsCode(type='code_interpreter')])

申し訳ございませんが、翻訳すべきテキストが提供されていないようです。「And the title is:」の後に続く内容が見当たりません。

翻訳したいテキストを完全に提供していただけますでしょうか？技術文書の内容を正確に翻訳いたします。

In [15]:
#Wait as assistant may take a few steps
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)


"Ascending Profits & Digital Dominance"


## 3. DALL·E-3 タイトル画像

素晴らしい、これでタイトル、プロット、そして2つの箇条書きができました。これらすべてをスライドに配置する準備がほぼ整いましたが、最後のステップとして、DALL·E-3にプレゼンテーションのタイトルスライドに使用する画像を作成してもらいましょう。<br><br>
*注意:* DALL·E-3はまだassistants API内では利用できませんが、近日中に利用可能になる予定です！<br><br>
私たちの会社（NotRealCorp）の簡単な説明を入力し、残りはDALL·E-3にお任せします！

In [16]:
company_summary = "NotReal Corp is a prominent hardware company that manufactures and sells processors, graphics cards and other essential computer hardware."


In [17]:
response = client.images.generate(
  model='dall-e-3',
  prompt=f"given this company summary {company_summary}, create an inspirational \
    photo showing the growth and path forward. This will be used at a quarterly\
       financial planning meeting",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url


素晴らしい、これでこの画像をスレッドに追加できます。まず、画像をローカルに保存し、次に`File`アップロードエンドポイントを使用してアシスタントAPIにアップロードします。画像も確認してみましょう。

In [18]:
dalle_img_path = '../images/dalle_image.png'
img = requests.get(image_url)

#Save locally
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#Upload
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)


![Image](../images/dalle_image.png)

## 4. スライドの作成

スライドを作成するために必要なコンテンツがすべて揃いました。単純にスライドを求めるメッセージを追加することもできますが、代わりに`python-pptx`ライブラリを使用したスライドテンプレートをアシスタントに提供しましょう。これにより、希望するスタイルのデッキを確実に取得できます。テンプレートの作成に関する注意事項については、ノートブックの最後にある`Extensions`セクションを参照してください。

In [19]:
title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left side of the slide with a margin at the top and bottom
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text box positioned higher
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle text box
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Define placeholders
image_path = data_vis_img
title_text = "Maximizing Profits: The Dominance of Online Sales & Direct Sales Optimization"
bullet_points = "• Online Sales consistently lead in profitability across quarters, indicating a strong digital market presence.\n• Direct Sales show fluctuations, suggesting variable performance and the need for targeted improvements in that channel."

# Add image placeholder on the left side of the slide
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text spanning the whole width
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add hardcoded "Key Insights" text and bullet points
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""



スライド用にいくつかの簡単な変数を設定しましょう。タイトルスライドには会社名のNotRealCorpを表示し、プレゼンテーションのタイトルは「Quarterly financial planning meeting, Q3, 2023」にします。

In [20]:
title_text = "NotRealCorp"
subtitle_text = "Quarterly financial planning meeting, Q3 2023"


そして、データスライドについては、以下があります：

ここでは、タイトルスライドを作成するためのテンプレートがあります。以下のテンプレートは、望ましいタイトルスライドの画像をGPT-Vにアップロードし、そのテンプレートを作成するための`python-pptx`コードを要求することで作成されました。テンプレートへの入力は、image_path、title_text、subtitle_textです。

In [21]:
submit_message(assistant.id,thread,f"Use the included code template to create a PPTX slide that follows the template format, but uses the image, company name/title, and document name/subtitle included:\
{title_template}. IMPORTANT: Use the image file included in this message as the image_path image in this first slide, and use the Company Name {title_text} as the title_text variable, and \
  use the subtitle_text {subtitle_text} a the subtitle_text variable. \
    NEST, create a SECOND slide using the following code template: {data_vis_template} to create a PPTX slide that follows the template format, but uses the company name/title, and document name/subtitle included:\
{data_vis_template}. IMPORTANT: Use the line plot image, that is the second attached image in this message, that you created earlier in the thread as the data_vis_img image, and use the data visualization title that you created earlier for the variable title_text, and\
  the bullet points of insights you created earlier for the bullet_points variable. Output these TWO SLIDES as a .pptx file. Make sure the output is two slides, with each slide matching the respective template given in this message.",
              file_ids=[dalle_file.id, plot_file.id]
)


Run(id='run_taLrnOnlDhoywgQFFBOLPlg0', assistant_id='asst_3T362kLlTyAq0FUnkvjjQczO', cancelled_at=None, completed_at=None, created_at=1701827118, expires_at=1701827718, failed_at=None, file_ids=['file-piTokyHGllwGITzIpoG8dok3'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_73TgtFoJMlEJvb13ngjTnAo3', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [22]:
#May take 1-3 mins
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)
        break
    except Exception as e:
        print("Assistant still working on PPTX...")
        time.sleep(10)


Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Successfully retrieved pptx_id: file-oa0i63qPH4IaJXYj90aA6L4Q


In [25]:
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("data/created_slides.pptx", "wb") as f:
    f.write(file_obj.getbuffer())


これで、作成したすべてのコンテンツが保存されたPPTXファイルができました！<br>

先ほどAssistants APIとDALL·E-3のみを使用して作成した.pptxファイルのスクリーンショットを見てみましょう。Assistants APIにはまだ`seed`パラメータがないため、LLMの非決定性により、このノートブックを実行した際に表示されるものとは、DALL·E-3の画像や文言が若干異なる場合がありますが、出力結果は方向性として同じものになるはずです。

タイトルスライド：

![Title Slide](../images/title_slide.png)

そして、データスライド：

![Data Slide](../images/data_vis_slide.png)

## 5. 結論

わあ！これらのスライドはフォーマットの調整が必要かもしれませんが、Assistants API、GPT-4、DALL·E-3を使用して素晴らしいコンテンツを作成することができました。財務データが含まれた`.csv`ファイルを取り込み、アシスタントを使用して流通チャネル別の四半期利益を計算し、結果をプロットし、可視化からインサイトと重要なポイントを特定し、要約的なタイトルを作成することができました。そして、私たちの会社NotRealCorpの説明だけを与えて、DALL·E-3を使用して素晴らしいタイトル画像を作成しました。<br><br>
人間が関与しないでこのプロセスを完全に自動化するにはまだ道のりがありますが、このノートブックがスライド作成プロセスを少しでも簡単にできることを願っています。さらに重要なことは、このノートブックがAssistants APIの可能性を垣間見せてくれることです！皆さんが何を構築するか楽しみにしています。

## 6. 拡張機能

- DALL·E-3がAssistants APIに組み込まれた際には、スレッド内で生成されたタイトル画像をリクエストする機能を利用できるようになります。
- GPT-4-VisionはまだAssistants APIでサポートされていませんが、折れ線グラフ画像から洞察を収集するために使用できたでしょう。
- GPT-4-Visionは、このレシピに含まれる`python-pptx`テンプレートの生成に使用されたため、画像をスライドテンプレートに変換する際のベストプラクティスを実演する拡張プロジェクトが考えられます。